In [ ]:
!pip install pycryptodome

In [ ]:
# based on https://stackoverflow.com/questions/39928401/recover-db-password-stored-in-my-dbeaver-connection

# requires pycryptodome lib (pip install pycryptodome)

import sys
import base64
import os
import json
from Crypto.Cipher import AES

default_paths = [
  '~/Library/DBeaverData/workspace6/General/.dbeaver/',
  '~/.local/share/DBeaverData/workspace6/General/.dbeaver/',
  '~/.local/share/.DBeaverData/workspace6/General/.dbeaver/',
  '~/AppData/Roaming/DBeaverData/workspace6/General/.dbeaver/',
]

cred_file = 'credentials-config.json'
conns_file = 'data-sources.json'


for path in default_paths:    
    cred_path = os.path.join(path, cred_file)
    conns_path = os.path.join(path, conns_file)
    filepath = os.path.expanduser(cred_path)
try:
    f = open(filepath, 'rb')
    f.close()    
except Exception as e:
    pass

#actual key link
#https://github.com/dbeaver/dbeaver/blob/8ed6d430a12ddc065a8bb69ec11babb152462f15/plugins/org.jkiss.dbeaver.registry/src/org/jkiss/dbeaver/registry/BaseProjectImpl.java#L77
PASSWORD_DECRYPTION_ORIGINAL_KEY = [-70, -69, 74, -97, 119, 74, -72, 83, -55, 108, 45, 101, 61, -2, 84, 74]
PASSWORD_DECRYPTION_KEY = bytes([i if i >= 0 else 256 + i for i in PASSWORD_DECRYPTION_ORIGINAL_KEY])

with open(filepath, 'rb') as f:
    data = f.read()

decryptor = AES.new(PASSWORD_DECRYPTION_KEY, AES.MODE_CBC, data[:16])
padded_output = decryptor.decrypt(data[16:])
output = padded_output.rstrip(padded_output[-1:])
json_creds = json.loads(output)

with open(os.path.expanduser(conns_path), 'r') as f:
    json_conns = json.load(f)['connections']

for k,v in json_conns.items():        
    name = v['configuration'].get('name')
    host = v['configuration'].get('host')
    database = v['configuration'].get('database')
    url = v['configuration'].get('url')
    port = v['configuration'].get('port')
    if k in json_creds.keys():
        json_creds[k]['#connection']['name'] = name
        json_creds[k]['#connection']['host'] = host
        json_creds[k]['#connection']['database'] = database
        json_creds[k]['#connection']['url'] = url
        json_creds[k]['#connection']['port'] = port    
    
try:
    print(json.dumps(json_creds, indent=4, sort_keys=True))
except:
    print(output)